# 1. Imports

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns

sns.set(style = 'white',context = 'notebook', palette = 'muted')

# 2.1 Data

In [4]:
train_file = '/Users/jerryyileibao/Downloads/titanic project/train.csv'
test_file = '/Users/jerryyileibao/Downloads/titanic project/test.csv'

train = pd.read_csv(train_file)
test = pd.read_csv(test_file)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/jerryyileibao/Downloads/train.csv'

# 2.2 Check shape

In [ ]:
print('train shape: ',train.shape)
print('test shape: ',test.shape)

#difference in num of columns lie in 'Survived'

# 2.3 Append into full dataset

In [ ]:
full = train.append(test,ignore_index=True)

# 3.1 Check info

In [ ]:
full.info()

# age missing some
# fare missing 1
# cabin missing a lot
# embarked missing 2

# 3.2.1 Plot the survival rate vs Embarked location

In [ ]:
sns.barplot(data=train,x='Embarked',y='Survived')

Those embarked from Cherbourg have the higest survival rate

# 3.2.2 Plot the survival rate vs parch (# of parent/children)

In [ ]:
sns.barplot(data=train,x='Parch',y='Survived')

Parch=3 (3 parents/children) group has the highest survival rate

# 3.2.3 Plot the survival rate vs SibSp (# of siblings / spouses)

In [ ]:
sns.barplot(data = train, x = 'SibSp', y = 'Survived')

Those with 1 or 2 siblings/spouses have the highest survival rate

# 3.2.4 Plot the survival rate vs Pclass

sns.barplot(data = train, x = 'Pclass', y = 'Survived')

Class 1 has the highest survival rate

# 3.2.5 Plot the survival rate vs gender

In [ ]:
sns.barplot(data = train, x = 'Sex', y = 'Survived')

Female are more likely to survive

# 3.2.6 Plot the survival rate vs age

In [ ]:
ageFacet=sns.FacetGrid(train,hue='Survived',aspect=3)
ageFacet.map(sns.kdeplot,'Age',shade=True)
ageFacet.set(xlim=(0,train['Age'].max()))
ageFacet.add_legend()

# 3.2.7 Plot the survival rate vs fare

In [ ]:
fareFacet = sns.FacetGrid(train,hue = 'Survived', aspect = 3)
fareFacet.map(sns.kdeplot,'Fare',shade = True)
fareFacet.set(xlim=(0,150))
fareFacet.add_legend()

In [ ]:
## Check the distribution of fare

In [ ]:
fare = sns.distplot(full['Fare'][full['Fare'].notnull()],label = 'skewness:%.2f'%(full['Fare'].skew()))
fare.legend()

In [ ]:
## Since fare is very skewed to the right, we use log value to solve the problem of uneven distribution

In [ ]:
full['Fare'] = full['Fare'].map(lambda x: np.log(x) if x>0 else 0)

# 4. Pre-processing

# 4.1 Data cleaning

## 4.1.1 Fill Cabin null value with NaN

In [ ]:
full['Cabin']=full['Cabin'].fillna('U')

In [ ]:
full['Cabin'].head()

## 4.1.2 Fill Embarked null value 

In [ ]:
full[full['Embarked'].isnull()]

In [ ]:
full['Embarked'].value_counts()

Since most people embarked from Southampton, we fill these two entries with Southampton

In [ ]:
full['Embarked']=full['Embarked'].fillna('S')

## 4.1.3 Fill Fare null value

In [ ]:
full[full['Fare'] == min(full['Fare'])]

In [ ]:
full[(full['Fare']==0)&(full['Pclass']==3)]['Fare'] = full[(full['Pclass']==3)&(full['Embarked']=='S')&(full['Fare']!=0)]['Fare'].mean()
full[(full['Fare']==0)&(full['Pclass']==2)]['Fare'] = full[(full['Pclass']==2)&(full['Embarked']=='S')&(full['Fare']!=0)]['Fare'].mean()
full[(full['Fare']==0)&(full['Pclass']==1)]['Fare'] = full[(full['Pclass']==1)&(full['Embarked']=='S')&(full['Fare']!=0)]['Fare'].mean()

# 4.2 Feature Engineering

## 4.2.1 Title in the name

In [ ]:
full['Title']=full['Name'].map(lambda x:x.split(',')[1].split('.')[0].strip())
full['Title'].value_counts()

In [ ]:
# Categorize the title
TitleDict={}
TitleDict['Mr']='Mr'
TitleDict['Mlle']='Miss'
TitleDict['Miss']='Miss'
TitleDict['Master']='Master'
TitleDict['Jonkheer']='Master'
TitleDict['Mme']='Mrs'
TitleDict['Ms']='Mrs'
TitleDict['Mrs']='Mrs'
TitleDict['Don']='Royalty'
TitleDict['Sir']='Royalty'
TitleDict['the Countess']='Royalty'
TitleDict['Dona']='Royalty'
TitleDict['Lady']='Royalty'
TitleDict['Capt']='Officer'
TitleDict['Col']='Officer'
TitleDict['Major']='Officer'
TitleDict['Dr']='Officer'
TitleDict['Rev']='Officer'

full['Title']=full['Title'].map(TitleDict)
full['Title'].value_counts()

In [ ]:
# visualize survival rate vs title
sns.barplot(data=full,x='Title',y='Survived')

We see that survival rate for "Mr" and "Officer" is significantly lower than other titles

## 4.2.2 Family Size

In [ ]:
#calculate the family size
full['familyNum']=full['Parch']+full['SibSp']+1

In [ ]:
#function to categorize different family sizes
def familysize(familyNum):
    if familyNum==1:
        return 0
    elif (familyNum>=2)&(familyNum<=4):
        return 1
    else:
        return 2

full['familySize']=full['familyNum'].map(familysize)
full['familySize'].value_counts()

In [ ]:
sns.barplot(data=full,x='familySize',y='Survived')

Survival is higest when family size is moderate

## 4.2.3 Cabin types

In [ ]:
#using the cabin starting letter to classify different cabins
full['Deck']=full['Cabin'].map(lambda x:x[0])

In [ ]:
sns.barplot(data=full,x='Deck',y='Survived')

B,D,E Cabins have higher survival rate; U and T have lower survival rate

## 4.2.4 Ticket number grouping

In [ ]:
full['Ticket']

In [ ]:
#extract the count of tickets with the same ticket number

TickCountDict={}
TickCountDict=full['Ticket'].value_counts()
TickCountDict.head()

In [ ]:
full['TicketCount'] = full['Ticket'].map(TickCountDict)
full['TicketCount'].head()

In [ ]:
sns.barplot(data=full,x='TicketCount',y='Survived')

In [ ]:
# group the passengers by count of tickets with the same ticket number
def TickCountGroup(num):
    if (num>=2)&(num<=4):
        return 0
    elif (num==1)|((num>=5)&(num<=8)):
        return 1
    else :
        return 2

full['TickGroup']=full['TicketCount'].map(TickCountGroup)

sns.barplot(data=full,x='TickGroup',y='Survived')

## 4.2.5 Age missing values

In [ ]:
full['Age'].isnull()

In [ ]:
full['Age'].isnull().sum()

Many rows don't have age, so we need to build a model to fill in the predicted age

In [ ]:
# filter the relevant vars
AgePre=full[['Age','Parch','Pclass','SibSp','Title','familyNum','TicketCount']]

#one-hot encoding
AgePre=pd.get_dummies(AgePre)
ParAge=pd.get_dummies(AgePre['Parch'],prefix='Parch')
SibAge=pd.get_dummies(AgePre['SibSp'],prefix='SibSp')
PclAge=pd.get_dummies(AgePre['Pclass'],prefix='Pclass')

In [ ]:
#check correlation
AgeCorrDf=pd.DataFrame()
AgeCorrDf=AgePre.corr()
AgeCorrDf['Age'].sort_values()

In [ ]:
#concatenate the one-hot encoding results
AgePre=pd.concat([AgePre,ParAge,SibAge,PclAge],axis=1)
AgePre.head()

In [ ]:
#train test split for random forest regression
AgeKnown=AgePre[AgePre['Age'].notnull()]
AgeUnKnown=AgePre[AgePre['Age'].isnull()]

#train split for x and y
AgeKnown_X=AgeKnown.drop(['Age'],axis=1)
AgeKnown_y=AgeKnown['Age']

#test data, get x
AgeUnKnown_X=AgeUnKnown.drop(['Age'],axis=1)

#model
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(random_state=None,n_estimators=500,n_jobs=-1)
rfr.fit(AgeKnown_X,AgeKnown_y)

In [ ]:
#model score
rfr.score(AgeKnown_X,AgeKnown_y)

In [ ]:
#predict the unknown age
AgeUnKnown_y=rfr.predict(AgeUnKnown_X)

#fill the unknown ages in full
full.loc[full['Age'].isnull(),['Age']]=AgeUnKnown_y

# 4.3 Potential peer effects 

In [ ]:
#We try to see if Surname has a peer effect, namely people with the same surname (i.e. relatives) have the tendency of surviving or dying together
#Since survival is closely relate to age and gender, we look at two specific groups:
#1. male above age 12 who survived
#2. female and children below 12 who did not survive

In [ ]:
#extract the surnames and their frequency
full['Surname']=full['Name'].map(lambda x:x.split(',')[0].strip())
SurNameDict={}
SurNameDict=full['Surname'].value_counts()
full['SurnameNum']=full['Surname'].map(SurNameDict)

In [ ]:
#divde into 2 groups
MaleDf=full[(full['Sex']=='male')&(full['Age']>12)&(full['familyNum']>=2)]
FemChildDf=full[((full['Sex']=='female')|(full['Age']<=12))&(full['familyNum']>=2)]

In [ ]:
#check out male survival rate by Surname
MSurNamDf=MaleDf['Survived'].groupby(MaleDf['Surname']).mean()
MSurNamDf.head()
MSurNamDf.value_counts()

We see strong peer effects. We will alter some feature values for male group that survived to increase their probability of being marked as survived

In [ ]:
#Get the dictionary of Surnames that 
MSurNamDict={}
MSurNamDict=MSurNamDf[MSurNamDf.values==1].index
MSurNamDict


In [ ]:
#same analysis for females and children group
FCSurNamDf=FemChildDf['Survived'].groupby(FemChildDf['Surname']).mean()
FCSurNamDf.head()
FCSurNamDf.value_counts()

In [ ]:
FCSurNamDict={}
FCSurNamDict=FCSurNamDf[FCSurNamDf.values==0].index
FCSurNamDict

Similarly, if we change these that did not survive into the male not survived group, then from the last group we see that male over 12 are less likely to survive. 
We do the same thing to the survived men. We see that women and children in this group are more likely to survive, so we modify the survived men to match the feature in this group.

In [ ]:
# Change the men into age under 12 and female
full.loc[(full['Survived'].isnull())&(full['Surname'].isin(MSurNamDict))&(full['Sex']=='male'),'Age']=6
full.loc[(full['Survived'].isnull())&(full['Surname'].isin(MSurNamDict))&(full['Sex']=='male'),'Sex']='female'

In [ ]:
#change the women into age over 12 and male
full.loc[(full['Survived'].isnull())&(full['Surname'].isin(FCSurNamDict))&((full['Sex']=='female')|(full['Age']<=12)),'Age']=50
full.loc[(full['Survived'].isnull())&(full['Surname'].isin(FCSurNamDict))&((full['Sex']=='female')|(full['Age']<=12)),'Sex']='male'

# Feature selection

Only keep the features that are highly correlated with Survived

In [ ]:
#drop the features that are not related
fullSel=full.drop(['Cabin','Name','Ticket','PassengerId','Surname','SurnameNum'],axis=1)

In [ ]:
#check the correlations between survived and other variables
corrDf=pd.DataFrame()
corrDf=fullSel.corr()
corrDf['Survived'].sort_values(ascending=True)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,8))
sns.heatmap(fullSel.corr(),cmap='BrBG',annot=True,
           linewidths=.5)
plt.xticks(rotation=45)

In [ ]:
#drop the low correlation features
fullSel=fullSel.drop(['familyNum','SibSp','TicketCount','Parch'],axis=1)

#one-hot encoding
fullSel=pd.get_dummies(fullSel)
PclassDf=pd.get_dummies(full['Pclass'],prefix='Pclass')
TickGroupDf=pd.get_dummies(full['TickGroup'],prefix='TickGroup')
familySizeDf=pd.get_dummies(full['familySize'],prefix='familySize')

In [ ]:
fullSel=pd.concat([fullSel,PclassDf,TickGroupDf,familySizeDf],axis=1)

# 5. Model

## 5.1 Model construction

Models include: Decision Tree, Gradient Boosting, Random Forest, KNN, Logistic Regression, SVC

In [ ]:
#train test split
train = fullSel[fullSel['Survived'].notnull()]
test = fullSel[fullSel['Survived'].isnull()]

In [ ]:
#train test x y
train_x = train.drop('Survived',axis=1)
train_y= train['Survived']

test_x = test.drop('Survived',axis=1)

In [ ]:
#imports
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold

In [ ]:
kfold=StratifiedKFold(n_splits=10)

In [ ]:
classifiers=[]
classifiers.append(SVC())
classifiers.append(DecisionTreeClassifier())
classifiers.append(RandomForestClassifier())
classifiers.append(GradientBoostingClassifier())
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression())

In [ ]:
#obtain resuls
cv_results = []
for classifier in classifiers:
    cv_results.append(cross_val_score(classifier,train_x,train_y,scoring = 'accuracy', cv = kfold,n_jobs = -1))

In [ ]:
#obtain mean and std
cv_means=[]
cv_std=[]

for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())

In [ ]:
#turn results into a dataframe
cvResDf=pd.DataFrame({'cv_mean':cv_means,
                     'cv_std':cv_std,
                     'algorithm':['SVC','DecisionTreeCla','RandomForestCla',
                                  'GradientBoostingCla','KNN','LR']})

In [ ]:
cvResDf

LR and GradientBoosting have the best performance

In [ ]:
#GBC
GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

In [ ]:
modelgsGBC = GridSearchCV(GBC,param_grid = gb_param_grid, cv=kfold, 
                                     scoring="accuracy", n_jobs= -1, verbose = 1)

In [ ]:
modelgsGBC.fit(train_x,train_y)

In [ ]:
#LR
modelLR=LogisticRegression()
LR_param_grid = {'C' : [1,2,3],
                'penalty':['l1','l2']}
modelgsLR = GridSearchCV(modelLR,param_grid = LR_param_grid, cv=kfold, 
                                     scoring="accuracy", n_jobs= -1, verbose = 1)
modelgsLR.fit(train_x,train_y)

## 5.2 Model score

In [ ]:
#modelgsGBC
print('modelgsGBC score：%.3f'%modelgsGBC.best_score_)
#modelgsLR模型
print('modelgsLR score：%.3f'%modelgsLR.best_score_)

In [ ]:
#ROC curve
#GBC
GBC_train_y = modelgsGBC.predict(train_x).astype(int)

from sklearn.metrics import roc_curve,auc
fpr,tpr,threshold = roc_curve(train_y,GBC_train_y)
roc_auc = auc(fpr,tpr)

plt.figure()
plt.figure(figsize = (10,10))
plt.plot(fpr,tpr,color = 'r', lw=2, label = 'ROC curve (area = %0.3f)'%roc_auc)
plt.plot([0,1], [0,1], color = 'navy', lw = 2, linestyle = '--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Titanic GradientBoosting Classifier Model')
plt.legend(loc = 'lower right')
plt.show()

LR_train_y = modelgsLR.predict(train_x).astype(int)

fpr,tpr,threshold = roc_curve(train_y, LR_train_y)
roc_auc = auc(fpr,tpr)
plt.figure()
plt.figure(figsize = (10,10))
plt.plot(fpr,tpr,color = 'r',lw = 2, label = 'ROC curve (area = %0.3f)'%roc_auc)
plt.plot([0,1], [0,1], color = 'navy', lw = 2, linestyle = '--')
plt.xlim([0.0,1.0])
plt.ylim([0.0,1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Titanic Logistic Regression Classifier Model')
plt.legend(loc = 'lower right')
plt.show()

GBC model performs better than LR

In [ ]:
#Confusion matrix

from sklearn.metrics import confusion_matrix
print('GBC confusion matrix \n',confusion_matrix(train_y.astype(int).astype(str),GBC_train_y.astype(int).astype(str)))
print('LR confusion matrix \n',confusion_matrix(train_y.astype(int).astype(str),LR_train_y.astype(int).astype(str)))

In [ ]:
tn, fp, fn, tp = confusion_matrix(train_y.astype(int).astype(str),GBC_train_y.astype(int).astype(str)).ravel()

In [ ]:
print('GBC precision = ',tp/(tp+fp))
print('GBC specificity = ',tn/(tn+fp))

In [ ]:
confusion_matrix(train_y.astype(int).astype(str),GBC_train_y.astype(int).astype(str)).ravel()

In [ ]:
tn, fp, fn, tp = confusion_matrix(train_y.astype(int).astype(str),LR_train_y.astype(int).astype(str)).ravel()
print('LR precision = ',tp/(tp+fp))
print('LR specificity = ',tn/(tn+fp))

GBC has a higher precision, meaning that of all predicted survived cases, 85% are actual survivals
GBC has a higher specificity, meaning that of all deaths, 91% are correctly identified by the model

# Prediction with GBC

In [ ]:
test_y = modelgsGBC.predict(test_x).astype(int)

In [ ]:
#export result

result = pd.DataFrame()
result['PassengerId']=full['PassengerId'][full['Survived'].isnull()]
result['Survived'] = test_y
result.to_csv('/Users/jerryyileibao/Downloads/titanic_result.csv',index = False)